### **Класифікація тексту для виявлення шахрайства**

**Завдання:**

Запропонувати та реалізувати Proof of Concept (PoC) рішення, яке буде класифікувати текстові описи користувачів із поля **"Про себе"** та виявляти потенційний шахрайський, неприйнятний контент або контактну інформацію.

1. Провести дослідження датасету: переглянути приклади текстів, щоб зрозуміти, як виглядає неприйнятний контент або що може вважатися контактною інформацією.
2. Запропонувати рішення: підібрати метод, який буде ефективним для класифікації тексту. Це може бути проста модель на основі обробки тексту або щось складніше, як нейронна мережа.
3. Реалізувати модель: вибрати та налаштувати модель для класифікації тексту. 
4. Оцінка результату: протестувати модель на наданих даних і перевірити, наскільки добре вона розрізняє нормальні тексти та порушення. 

**Деталі:**

Датасет для цього завдання доступний за [посиланням.](https://docs.google.com/spreadsheets/d/1FKPBf_-D2UbB6zij8snVqqg02WKhMb8lZFhiOj1L6tU/edit?usp=sharing) Вам надано датасет, який містить текстові описи з поля "Про себе" та мітки:

- **label**: позначає, чи є текст неприйнятним:
    - 1 — неприйнятний контент.
    - 0 — нормальний контент.
- **contact_details**: додаткова колонка, яку можна використати для детектування полів, що містять контактну інформацію користувачів.

Дана задача включає дві підзадачі:
1. **Детектування контактної інформації** у полі "Про себе" (телефон, email, посилання).
2. **Виявлення іншого неприйнятного контенту** (підозрілі тексти, нецензурна лексика тощо).

Обидві задачі можна вирішити як окремими моделями, так і спільним рішенням, проте важливо забезпечити здатність чітко розрізняти ці два класи.

### Завантаження даних

Завантажимо датасет та переглянимо як виглядають записи

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('data\Scam _ Not scam 200 - Sheet1.csv')
df.head()

,about_me,label,contact details
0,A︇︆︈l︇︆︈l︇︆︈ ︇︆︈m︇︆︈y︇︆︈ ︇︆︈s︇︆︈e︇︆︈x︇︆︈y︇︆︈ ︇...,1,1
1,solo busco que me traten como una princesa y m...,1,1
2,buenas tardes yo busco conocer personas yo qui...,1,0
3,Tex бO78OѲ4らч8,1,1
4,"soy madre soltera tengo un hijo de 13 años, so...",1,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   about_me         200 non-null    object
 1   label            200 non-null    int64 
 2   contact details  200 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ KB


In [12]:
df.label.value_counts()

label
0    104
1     96
Name: count, dtype: int64

In [7]:
df['contact details'].value_counts()

contact details
0    152
1     48
Name: count, dtype: int64

Датасет містить 3 колонки та 200 записів, і не має пропущених значень

Подивимось як виглядають дані які можна вважати прийнятними та не прийнятними

In [17]:
df[df.label == 1].head(10)

,about_me,label,contact details
0,A︇︆︈l︇︆︈l︇︆︈ ︇︆︈m︇︆︈y︇︆︈ ︇︆︈s︇︆︈e︇︆︈x︇︆︈y︇︆︈ ︇...,1,1
1,solo busco que me traten como una princesa y m...,1,1
2,buenas tardes yo busco conocer personas yo qui...,1,0
3,Tex бO78OѲ4らч8,1,1
4,"soy madre soltera tengo un hijo de 13 años, so...",1,0
5,Мне 16 если хочешь познакомиться пиши мне в Ин...,1,1
6,"bin für ein Sextreffen zu haben, wenn du Inter...",1,0
7,me gustan los hombre mayores y si es posible q...,1,0
8,Ꮥር.emilyqed,1,1
9,"люблю музику і малювати пхпх, можливо не вигля...",1,0


In [15]:
df[df.label == 0].head(10)


,about_me,label,contact details
40,"Люблю спілкуватись, багато сміятись. Проста, в...",0,0
41,Живу в Алматы. 🇰🇿 Мужчины рада видеть вас счас...,0,0
44,busco relación de seria,0,0
45,amante ala música salsa y ala coxinar.,0,0
46,Hay I'm a professional in basketball trying to...,0,0
47,just looking to have fun with some cool people,0,0
48,"Soy una mujer que sabe respetar, con un buen s...",0,0
49,"Lo más importante en la vida es la honestidad,...",0,0
50,"Olá, sou uma pessoa que gosto de coisas simple...",0,0
51,I'm a loving n caring person who enjoys cookin...,0,0


Тепер подивимось на дані що містять контакту інформацію

In [18]:
df[df['contact details'] == 1].head(10)


,about_me,label,contact details
0,A︇︆︈l︇︆︈l︇︆︈ ︇︆︈m︇︆︈y︇︆︈ ︇︆︈s︇︆︈e︇︆︈x︇︆︈y︇︆︈ ︇...,1,1
1,solo busco que me traten como una princesa y m...,1,1
3,Tex бO78OѲ4らч8,1,1
5,Мне 16 если хочешь познакомиться пиши мне в Ин...,1,1
8,Ꮥር.emilyqed,1,1
10,"amo a mi familia, mis amigas y el amor sincero...",1,1
11,💋💋💋I am Angelina by name ... I'm new to this t...,1,1
12,"Soy muy divertida , sales only🍑rnC0ntenido $",1,1
13,Ig frankimeyyrnalguien que me consienta,1,1
14,M︈︇︉y︈︇︉ ︈︇︉r︈︇︉e︈︇︉a︈︇︉l︈︇︉ ︈︇︉s︈︇︉e︈︇︉x︈︇︉y︈...,1,1


якщо є контакти це завжди неприйнятно?
як опрацьовувати різні мови?

ідентифікувати мови і розділити дані?
графіки розподілу по мовам, по довжині текстів, по міткам, поєдніти щоб перевірити чи всі тексти в яких є контакти є неприйнятними


In [19]:
from sklearn.model_selection import train_test_split

In [ ]:
trai